In [18]:
from torch.utils.data import DataLoader, random_split
import torch
from torchvision import transforms
from PIL import Image
import os
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import pandas as pd
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import sys
sys.path.append('../src/')
from dataset_class import FoodRecipeDataset

In [2]:
import torchtext
torchtext.__version__

'0.16.0'

In [24]:
# read csv
df = pd.read_csv("../src/dataset/food.csv")
df

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."
...,...,...,...,...,...
13466,Brownie Pudding Cake,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Preheat the oven to 350°F. Into a bowl sift to...,brownie-pudding-cake-14408,"['1 cup all-purpose flour', '2/3 cup unsweeten..."
13467,Israeli Couscous with Roasted Butternut Squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Preheat oven to 475°F.\nHalve lemons and scoop...,israeli-couscous-with-roasted-butternut-squash...,"['1 preserved lemon', '1 1/2 pound butternut s..."
13468,Rice with Soy-Glazed Bonito Flakes and Sesame ...,['Leftover katsuo bushi (dried bonito flakes) ...,"If using katsuo bushi flakes from package, moi...",rice-with-soy-glazed-bonito-flakes-and-sesame-...,['Leftover katsuo bushi (dried bonito flakes) ...
13469,Spanakopita,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,Melt 1 tablespoon butter in a 12-inch heavy sk...,spanakopita-107344,['1 stick (1/2 cup) plus 1 tablespoon unsalted...


Load FoodRecipeDataset

In [28]:
tokenizer = get_tokenizer('basic_english')

current_working_directory = os.getcwd()
images_path = os.path.join(current_working_directory, "../src/dataset/Food Images")
text_path = os.path.join(current_working_directory, "../src/dataset/food.csv")

FoodRecipeData = FoodRecipeDataset(text_path, images_path, transform=transforms.ToTensor())

training_share = 0.8 #Proportion of data that is alotted to the training set
training_size = int(training_share*len(FoodRecipeData))
test_size = len(FoodRecipeData) - training_size
generator = torch.Generator().manual_seed(42)
training_data, temp = random_split(FoodRecipeData, [training_size, test_size], generator)
test_data, val_data = random_split(temp, [int(0.4*len(temp)), int(0.6*len(temp))], generator)

assert len(test_data)+len(val_data)+len(training_data) == len(df),  "Check partition of train, val, and test sets"

Build vocab from training data

In [73]:
def yield_tokens(data_iter):
    for _, title, ingredients, instructions, cleaned_ingredients in tqdm(data_iter):
        yield tokenizer(title)
        yield tokenizer(ingredients)
        yield tokenizer(instructions)
        yield tokenizer(cleaned_ingredients)

        
token_generator = yield_tokens(training_data)

# build vocab with special tokens <unk> <pad> <start> <end>
vocab = build_vocab_from_iterator(token_generator, specials=["<unk>", "<pad>", "<start>", "<end>"], min_freq=1)
vocab.set_default_index(vocab["<unk>"])

# save vocabulary
torch.save(vocab, 'dataset/vocab.pt')

  0%|          | 0/10776 [00:00<?, ?it/s]

In [30]:
vocab = torch.load("../src/dataset/vocab.pt")
vocab.get_itos()

['<unk>',
 '<pad>',
 '<start>',
 '<end>',
 "'",
 ',',
 '.',
 'and',
 '1',
 'to',
 'the',
 'a',
 '2',
 '(',
 ')',
 'cup',
 'in',
 'with',
 '1/2',
 'until',
 'salt',
 'of',
 'or',
 'teaspoon',
 'tablespoons',
 'minutes',
 'about',
 'oil',
 '1/4',
 'into',
 '3',
 'large',
 ']',
 '[',
 'cups',
 'chopped',
 'pepper',
 'add',
 'for',
 'heat',
 'sugar',
 '4',
 'fresh',
 'over',
 'bowl',
 'butter',
 'tablespoon',
 'on',
 'water',
 'cut',
 'medium',
 'ground',
 'garlic',
 'then',
 'juice',
 'finely',
 'baking',
 'teaspoons',
 'olive',
 'is',
 '3/4',
 'small',
 'from',
 'lemon',
 'pan',
 'at',
 'cook',
 '6',
 '5',
 'flour',
 'kosher',
 'mixture',
 'transfer',
 'sliced',
 'cream',
 'sauce',
 'ounces',
 'oven',
 'plus',
 'more',
 'chicken',
 'black',
 'remaining',
 'stir',
 'if',
 'let',
 '8',
 'red',
 'place',
 'leaves',
 'brown',
 'as',
 'onion',
 'cover',
 'divided',
 'stirring',
 'grated',
 'freshly',
 'unsalted',
 'it',
 'top',
 'cool',
 'each',
 'dough',
 'skillet',
 'can',
 'pieces',
 'vine

In [31]:
# view vocabulary
print("Number of elements in vocabulary:", len(vocab.get_stoi()))
sorted(vocab.get_stoi().items(), key=lambda x: x[1])

Number of elements in vocabulary: 21656


[('<unk>', 0),
 ('<pad>', 1),
 ('<start>', 2),
 ('<end>', 3),
 ("'", 4),
 (',', 5),
 ('.', 6),
 ('and', 7),
 ('1', 8),
 ('to', 9),
 ('the', 10),
 ('a', 11),
 ('2', 12),
 ('(', 13),
 (')', 14),
 ('cup', 15),
 ('in', 16),
 ('with', 17),
 ('1/2', 18),
 ('until', 19),
 ('salt', 20),
 ('of', 21),
 ('or', 22),
 ('teaspoon', 23),
 ('tablespoons', 24),
 ('minutes', 25),
 ('about', 26),
 ('oil', 27),
 ('1/4', 28),
 ('into', 29),
 ('3', 30),
 ('large', 31),
 (']', 32),
 ('[', 33),
 ('cups', 34),
 ('chopped', 35),
 ('pepper', 36),
 ('add', 37),
 ('for', 38),
 ('heat', 39),
 ('sugar', 40),
 ('4', 41),
 ('fresh', 42),
 ('over', 43),
 ('bowl', 44),
 ('butter', 45),
 ('tablespoon', 46),
 ('on', 47),
 ('water', 48),
 ('cut', 49),
 ('medium', 50),
 ('ground', 51),
 ('garlic', 52),
 ('then', 53),
 ('juice', 54),
 ('finely', 55),
 ('baking', 56),
 ('teaspoons', 57),
 ('olive', 58),
 ('is', 59),
 ('3/4', 60),
 ('small', 61),
 ('from', 62),
 ('lemon', 63),
 ('pan', 64),
 ('at', 65),
 ('cook', 66),
 ('6', 6

In [32]:
# test vocab
vocab(tokenizer("The elephant is missing"))

[10, 6694, 59, 0]

Make dataloader

In [35]:
current_working_directory = os.getcwd()
images_path = os.path.join(current_working_directory, "../src/dataset/Food Images")
text_path = os.path.join(current_working_directory, "../src/dataset/food.csv")


class VocabImageDataset(FoodRecipeDataset):
    def __init__(self, annotations_file, img_dir, vocab, tokenizer, max_seq_len=512, device='cpu', transform=None):
        self.recipe_text = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.vocab = vocab
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.device = device
        self.transform = transform
        
        self.text_pipeline = lambda x : vocab(tokenizer(x))
        
    def __len__(self):
        return len(self.recipe_text)
        
    def __getitem__(self, idx):
        img_path = str(os.path.join(self.img_dir, self.recipe_text.iloc[idx, 3]) + ".jpg")
        image = Image.open(img_path)
        title_seq = str(self.recipe_text.iloc[idx, 0])
        ingredients_seq = str(self.recipe_text.iloc[idx, 1])
        instructions_seq = str(self.recipe_text.iloc[idx, 2])
        cleaned_ingredients_seq = str(self.recipe_text.iloc[idx, 4])
        
        title = torch.tensor(self.text_pipeline(title_seq))[:self.max_seq_len]
        ingredients = torch.tensor(self.text_pipeline(ingredients_seq))[:self.max_seq_len]
        instructions = torch.tensor(self.text_pipeline(instructions_seq))[:self.max_seq_len]
        cleaned_ingredients = torch.tensor(self.text_pipeline(cleaned_ingredients_seq))[:self.max_seq_len]
        
        output_dict = {'image': self.transform(image).to(self.device), 'title': title.to(self.device), 'ingredients': ingredients.to(self.device),
                       'instructions': instructions.to(self.device), 'cleaned_ingredients': cleaned_ingredients.to(self.device)}
        
#         return output_dict
        return self.transform(image).to(self.device), title.to(self.device), ingredients.to(self.device), instructions.to(self.device), cleaned_ingredients.to(self.device)


        

In [36]:
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor() 
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

VocabImage = VocabImageDataset(annotations_file=text_path, img_dir=images_path, vocab=vocab, tokenizer=tokenizer, device=device, transform=image_transform)
training_share = 0.8 #Proportion of data that is alotted to the training set
training_size = int(training_share*len(VocabImage))
test_size = len(VocabImage) - training_size
generator = torch.Generator().manual_seed(42)
train_data, temp = random_split(VocabImage, [training_size, test_size], generator)
test_data, val_data = random_split(temp, [int(0.4*len(temp)), int(0.6*len(temp))], generator)

Dataloader and collate_fn

In [38]:
def pad_input(input):
    """
    creates a padded tensor to fit the longest sequence in the batch
    
    Adapted from: https://github.com/amzn/image-to-recipe-transformers/blob/96e257e910c79a5411c3f65f598dd818f72fc262/src/dataset.py#L123C1-L141C19
    """
    if len(input[0].size()) == 1:
        l = [len(elem) for elem in input]
        
        # get idx token corresponding to <pad> in vocab
        pad_token = vocab.get_stoi()['<pad>']
        
        targets = torch.zeros(len(input), max(l)).long()+pad_token
        for i, elem in enumerate(input):
            end = l[i]
            targets[i, :end] = elem[:end]
    else:
        n, l = [], []
        for elem in input:
            n.append(elem.size(0))
            l.append(elem.size(1))
            
        # get idx token corresponding to <pad> in vocab
        pad_token = vocab.get_stoi()['<pad>']
        
        
        targets = torch.zeros(len(input), max(n), max(l)).long()+pad_token
        for i, elem in enumerate(input):
            targets[i, :n[i], :l[i]] = elem
    return targets


In [39]:

def collate_batch(batch):
    
    image, title, ingredients, instructions, cleaned_ingredients = zip(*batch)
    
    image_tensor = torch.stack(image, dim=0)
    title_tensor = pad_input(title)
    ingredients_tensor = pad_input(ingredients)
    instructions_tensor = pad_input(instructions)
    cleaned_ingredients_tensor = pad_input(cleaned_ingredients)
    
    return image_tensor, title_tensor, ingredients_tensor, instructions_tensor, cleaned_ingredients_tensor
    
    
    
    

In [40]:
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_batch)
val_dataloader = DataLoader(val_data, batch_size=32, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True, collate_fn=collate_batch)

In [41]:
for image, title, ingredients, instructions, cleaned_ingredients in train_dataloader:
    print(image.shape)
    print(ingredients.shape)
    print(instructions.shape)
    print(cleaned_ingredients.shape)
    break

torch.Size([32, 3, 224, 224])
torch.Size([32, 280])
torch.Size([32, 512])
torch.Size([32, 276])
